# Extractive Summarizer

In [1]:
import nltk
from nltk.corpus import stopwords
from nltk.cluster.util import cosine_distance
import numpy as np
import networkx as nx

In [2]:
def read_func(filename):
    '''
    Takes the whole text file as input
    Returns list of list of words of every sentence
    '''
    file = open(filename,'r',encoding='utf-8') # open file
    filedata = file.readlines() # real all lines
    para = ' '.join(filedata) # join all lines together
    lines = nltk.sent_tokenize(para) # every line is different elements in list
    sentences = [] # nested list, sentences, containing list of words
    for line in lines: # iterating in lines
        sentences.append(line.split(" ")) # splitting lines into words to add into sentences
    for line in sentences: # iterating lines again to remove fullstop
        line[len(line)-1] = line[len(line)-1][:(len(line[len(line)-1])-1)] # removed full stops
    return sentences

In [3]:
def sentence_similarity(sent1, sent2, stopwords=[]):
    '''
    Takes 2 sentences at a time as input and also takes the ntlk stopwords
    Returns cosine similarity between the sentence vectors (1 - distance = similarity)
    '''
    sent1 = [word.lower() for word in sent1 if word.lower() not in stopwords] # converted to lower case and removed stop words
    sent2 = [word.lower() for word in sent2 if word.lower() not in stopwords]

    all_words = list(set(sent1 + sent2)) # get list of all words in the 2 sentences

    vector1 = [0] * len(all_words)
    vector2 = [0] * len(all_words)

    # build vector for first sentence
    for word in sent1:
        vector1[all_words.index(word)] += 1

    # build vector for second sentence
    for word in sent2:
        vector2[all_words.index(word)] += 1
    
    return 1 - cosine_distance(vector1, vector2)

In [4]:
def build_similarity_matrix(sentences, stopwords = []):
    '''
    Takes sentences and stopwords as input
    Makes a similarity matrix using sentence_similarity function
    Returns the similarity matrix
    '''
    similarity_matrix = np.zeros((len(sentences),len(sentences))) # Creates empty 2D matrix

    for index_1 in range(len(sentences)):
        for index_2 in range(len(sentences)):
            if index_1 == index_2: # ignore diagonal terms (where sentences are same)
                continue
            similarity_matrix[index_1][index_2] = sentence_similarity(sentences[index_1], sentences[index_2], stopwords)

    return similarity_matrix

In [5]:
def generate_summary(file_name, num_of_lines = 5):
    stop_words = stopwords.words('english')
    summarize_text = []

    # reads file and returns list of list of words of a sentence
    sentences = read_func(file_name)

    # builds a 2d matrix of cosine similarity between all sentences
    similarity_matrix = build_similarity_matrix(sentences,stop_words)

    sentence_similarity_graph = nx.from_numpy_array(similarity_matrix)
    scores = nx.pagerank(sentence_similarity_graph)

    ranked_sentence = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)    

    for i in range(num_of_lines):
        summarize_text.append(" ".join(ranked_sentence[i][1]))

    summarize_text.append('')

    print('Summarized Text:')
    summary = ". ".join(summarize_text)
    print(summary)

In [6]:
generate_summary('sample_text_1.txt',4)

Summarized Text:
Ashwin said the news of Hemil’s death reached them on February 23. “We humbly request our government to talk to the Russian authorities and get the body of my son to his home town Surat. He left for Russia on December 14, 2023 and remained in contact with us regularly,” said his father. A family source said Hemil left studies after Class 12 and started a small embroidery business with his cousins. 


In [7]:
generate_summary('sample_text_2.txt',4)

Summarized Text:
Also, you would be surprised... to my mind the BJP is going to be No 1 in Bengal," Mr Kishor said, flagging states in each of which the party has lost the most recent Assembly election. Mr Kishor - who has guided the Congress, the Trinamool, and the Aam Aadmi Party to wins over the Prime Minister's BJP - repeated his warnings of the past - that the ruling party is not invincible. Less than two weeks before the general election, poll strategist Prashant Kishor sounded a red alert for the Congress and INDIA opposition bloc, warning of improved performances - seats + vote share - by Prime Minister Narendra Modi's Bharatiya Janata Party across six eastern and southern states. Three of these six states - Bengal, Kerala, and Tamil Nadu - will vote by 2026. 
